# Run Multiple instances of EICrecon using podio over TCP
(n.b. This is based on the fabric iperf3 example.)

This will setup a node at CERN and several other nodes at:

New York
Washington D.C.
Atlanta
Dallas
Los Angeles
Seattle
Salt Lake City
Kansas City

This will demonstrate transferring podio events and processing them using the EICrecon software from ePIC. Each site will run 4 independent eicrecon processes for a total of 32 processes. All will be pulling events from the same CERN server. The processing rate of a single process is ~1Hz so <32Hz can be achieved with this.


## Import the FABlib Library


In [1]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,a7818636-1fa1-4e77-bb03-d171598b0862
Bastion Host,bastion.fabric-testbed.net
Bastion Username,davidl_0004580836
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet. 

Patience here. This will take a while not only to set up the slice, but to pull the docker image which is >=11.3GB

In [2]:
slice_name = 'EICreconTCPmulti'
# [site1, site2] = fablib.get_random_sites(count=2)
server = 'CERN'
workers = ['NEWY','WASH','ATLA','KANS','DALL','SALT','SEAT','LOSA']
print(f"Server  site: {server}")
print(f"Worker sites: {workers}")

node1_name='Node1'
node2_name='Node2'

Server  site: CERN
Worker sites: ['NEWY', 'WASH', 'ATLA', 'KANS', 'DALL', 'SALT', 'SEAT', 'LOSA']


In [3]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

# Server node
server_name = f"server_{server}"
print(f"setting up: {server_name}")
server_node = slice.add_node(name=server_name, cores=4, disk=50, ram=16, site=server, image='docker_rocky_8')
server_node.add_fabnet()
# server_node.add_post_boot_upload_directory('node_tools','.')
server_node.add_post_boot_upload_directory('/home/fabric/work/jupyter-examples-rel1.6.1/fabric_examples/complex_recipes/iPerf3/node_tools','.')
server_node.add_post_boot_execute('sudo node_tools/host_tune.sh')
server_node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
server_node.add_post_boot_execute('docker pull eicweb/jug_xl:nightly ')


# Worker nodes
worker_nodes = {}
for site in workers:
    worker_name = f"worker_{site}"
    print(f"setting up: {worker_name}")
    node = slice.add_node(name=worker_name, cores=4, disk=50, ram=16, site=site, image='docker_rocky_8')
    worker_nodes[site] = node
    node.add_fabnet()
    # node.add_post_boot_upload_directory('node_tools','.')
    node.add_post_boot_upload_directory('/home/fabric/work/jupyter-examples-rel1.6.1/fabric_examples/complex_recipes/iPerf3/node_tools','.')
    node.add_post_boot_execute('sudo node_tools/host_tune.sh')
    node.add_post_boot_execute('node_tools/enable_docker.sh {{ _self_.image }} ')
    node.add_post_boot_execute('docker pull eicweb/jug_xl:nightly ')

#Submit Slice Request
slice.submit();


Retry: 14, Time: 819 sec


ID,94bf79ce-7d0c-4e25-bc5e-7ddf8aa603e6
Name,EICreconTCPmulti
Lease Expiration (UTC),2024-04-15 11:54:13 +0000
Lease Start (UTC),2024-04-14 11:54:14 +0000
Project ID,a7818636-1fa1-4e77-bb03-d171598b0862
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
19bdb4d0-619a-4abe-8d98-6ad35a986b43,server_CERN,4,16,100,docker_rocky_8,qcow2,cern-w1.fabric-testbed.net,CERN,rocky,2001:400:a100:3090:f816:3eff:fe1f:f5e9,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe1f:f5e9,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
abc1026b-fc73-4ed7-9759-e37c29ded40b,worker_ATLA,4,16,100,docker_rocky_8,qcow2,atla-w1.fabric-testbed.net,ATLA,rocky,2001:400:a100:3050:f816:3eff:fe39:d991,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3050:f816:3eff:fe39:d991,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
eeb8ae21-e7c7-4cc1-8940-6cbe0fe9a3ba,worker_DALL,4,16,100,docker_rocky_8,qcow2,dall-w1.fabric-testbed.net,DALL,rocky,2001:400:a100:3000:f816:3eff:fe73:2f04,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3000:f816:3eff:fe73:2f04,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ac0936ca-b44e-4388-aa58-0a4c775234b9,worker_KANS,4,16,100,docker_rocky_8,qcow2,kans-w1.fabric-testbed.net,KANS,rocky,2001:400:a100:3060:f816:3eff:fed5:e8ae,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3060:f816:3eff:fed5:e8ae,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
710cdd4d-1c2a-4c6d-9126-3aadf1ecc076,worker_LOSA,4,16,100,docker_rocky_8,qcow2,losa-w2.fabric-testbed.net,LOSA,rocky,2001:400:a100:3070:f816:3eff:fe8a:e270,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3070:f816:3eff:fe8a:e270,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
1b3e6ea2-1f39-467e-9bf9-cd756356a0f7,worker_NEWY,4,16,100,docker_rocky_8,qcow2,newy-w1.fabric-testbed.net,NEWY,rocky,2001:400:a100:3040:f816:3eff:fe82:57a5,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3040:f816:3eff:fe82:57a5,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
709bf2f5-e684-4671-816e-042b50c94751,worker_SALT,4,16,100,docker_rocky_8,qcow2,salt-w2.fabric-testbed.net,SALT,rocky,2001:400:a100:3010:f816:3eff:fe26:3565,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3010:f816:3eff:fe26:3565,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
ff915fef-a6ee-466e-9917-8e53a158b6e8,worker_SEAT,4,16,100,docker_rocky_8,qcow2,seat-w2.fabric-testbed.net,SEAT,rocky,2001:400:a100:3080:f816:3eff:feb6:523c,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3080:f816:3eff:feb6:523c,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key
99cf0a22-0da4-4c9b-a6b3-dc0ede261950,worker_WASH,4,16,100,docker_rocky_8,qcow2,wash-w1.fabric-testbed.net,WASH,rocky,2001:400:a100:3020:f816:3eff:feeb:5ab4,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3020:f816:3eff:feeb:5ab4,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
82018963-a71e-4b51-9e97-2d7246d77313,FABNET_IPv4_ATLA,L3,FABNetv4,ATLA,10.138.134.0/24,10.138.134.1,Active,
4381cd79-65d8-4c76-8410-387658c980bd,FABNET_IPv4_CERN,L3,FABNetv4,CERN,10.143.4.0/24,10.143.4.1,Active,
8f5a134a-be4c-4dd6-834d-061265b1c01c,FABNET_IPv4_DALL,L3,FABNetv4,DALL,10.133.129.0/24,10.133.129.1,Active,
9983ac26-220a-4879-8303-6b5223b30492,FABNET_IPv4_KANS,L3,FABNetv4,KANS,10.138.1.0/24,10.138.1.1,Active,
c397ae4d-21f1-46e3-98d1-3f907e1e9cd6,FABNET_IPv4_LOSA,L3,FABNetv4,LOSA,10.137.5.0/24,10.137.5.1,Active,
3b10e239-897d-4272-9ed3-ba7991b729ce,FABNET_IPv4_NEWY,L3,FABNetv4,NEWY,10.137.132.0/24,10.137.132.1,Active,
83aaebb4-bfdb-43d5-9393-6f872986ba09,FABNET_IPv4_SALT,L3,FABNetv4,SALT,10.134.6.0/24,10.134.6.1,Active,
f1f164f8-4f1f-4b80-87f9-6823aa063905,FABNET_IPv4_SEAT,L3,FABNetv4,SEAT,10.139.1.0/24,10.139.1.1,Active,
17ebd3a6-aa33-4b50-8344-876aed847db3,FABNET_IPv4_WASH,L3,FABNetv4,WASH,10.133.7.0/24,10.133.7.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node
server_CERN-FABNET_IPv4_CERN_nic-p1,p1,server_CERN,FABNET_IPv4_CERN,100,auto,,0A:82:F3:50:5F:D6,enp7s0,enp7s0,10.143.4.2,6
worker_NEWY-FABNET_IPv4_NEWY_nic-p1,p1,worker_NEWY,FABNET_IPv4_NEWY,100,auto,,0A:85:1A:26:F1:8A,enp7s0,enp7s0,10.137.132.2,4
worker_WASH-FABNET_IPv4_WASH_nic-p1,p1,worker_WASH,FABNET_IPv4_WASH,100,auto,,92:82:14:27:38:84,enp7s0,enp7s0,10.133.7.2,6
worker_ATLA-FABNET_IPv4_ATLA_nic-p1,p1,worker_ATLA,FABNET_IPv4_ATLA,100,auto,,06:26:E4:0E:88:B4,enp7s0,enp7s0,10.138.134.2,4
worker_KANS-FABNET_IPv4_KANS_nic-p1,p1,worker_KANS,FABNET_IPv4_KANS,100,auto,,06:1F:6C:49:30:75,enp7s0,enp7s0,10.138.1.2,6
worker_DALL-FABNET_IPv4_DALL_nic-p1,p1,worker_DALL,FABNET_IPv4_DALL,100,auto,,32:33:2B:8D:32:84,enp7s0,enp7s0,10.133.129.2,6
worker_SALT-FABNET_IPv4_SALT_nic-p1,p1,worker_SALT,FABNET_IPv4_SALT,100,auto,,06:72:BC:E6:0F:21,enp7s0,enp7s0,10.134.6.2,4
worker_SEAT-FABNET_IPv4_SEAT_nic-p1,p1,worker_SEAT,FABNET_IPv4_SEAT,100,auto,,16:E2:69:28:D2:6E,enp7s0,enp7s0,10.139.1.2,4
worker_LOSA-FABNET_IPv4_LOSA_nic-p1,p1,worker_LOSA,FABNET_IPv4_LOSA,100,auto,,0A:4C:01:D1:D8:41,enp6s0,enp6s0,10.137.5.2,4



Time to print interfaces 819 seconds


# Build software

This will clone and build the software needed for the test. It is built on both nodes though really only podio2tcp is needed on one of them.

This will take quite a long time since a lot of code is compiled and only a couple of cores are available.

In [19]:
%cd /home/fabric/work/RTDP

slice = fablib.get_slice(slice_name)
server_node = slice.get_node(name=server_name)
for site,node in worker_nodes.items():
    worker_name = f"worker_{site}"
    node = slice.get_node(name=worker_name)
    worker_nodes[site] = node

cmd  = "docker run --rm "
cmd += "--network host "
cmd += "-v ${PWD}:/work "
cmd += "eicweb/jug_xl:nightly "
cmd += "/work/build_all.sh"

# Run builds in parallel and keep track of threads used for each
threads = [] 

# Run build on server node
server_node.upload_file('build_all.sh', 'build_all.sh')
server_node.execute("chmod +x build_all.sh", quiet=True)
print(f"starting build: {server_node}")
thr = server_node.execute_thread(cmd, output_file=f"{server_node.get_name()}.log");
threads.append(thr)

# Run build on all worker nodes
for site,node in worker_nodes.items():
    node.upload_file('build_all.sh', 'build_all.sh')
    node.execute("chmod +x build_all.sh", quiet=True)
    print(f"starting build: worker_{site}")
    thr = node.execute_thread(cmd, output_file=f"{node.get_name()}.log");
    threads.append(thr)

# Call result on all threads which I *think* will block on each until they are all done.
print(f"Nthreads started: {len(threads)}")
for thr in threads:
    thr.result()


/home/fabric/work/RTDP
starting build: -----------------  -------------------------------------------------------------------------------------------------------------------------------------------
ID                 19bdb4d0-619a-4abe-8d98-6ad35a986b43
Name               server_CERN
Cores              4
RAM                16
Disk               100
Image              docker_rocky_8
Image Type         qcow2
Host               cern-w1.fabric-testbed.net
Site               CERN
Management IP      2001:400:a100:3090:f816:3eff:fe1f:f5e9
Reservation State  Active
Error Message
SSH Command        ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config rocky@2001:400:a100:3090:f816:3eff:fe1f:f5e9
-----------------  -------------------------------------------------------------------------------------------------------------------------------------------
starting build: worker_NEWY
starting build: worker_WASH
starting build: worker_ATLA
starting build: work

# Copy input file to source host

TODO: The podiostr input file is currently copied from my bastion account. It would be better to have this pulled from xrootd by the remote nodes.

The input file was copied to my bastion account with:

~~~
  scp -J davidl@scilogin.jlab.org davidl@ifarm9:/home/davidl/work_eic/2024.04.11.podio_stream/simout.1000.edmhep.root.podiostr
~~~

Instructions for creating it can be found here: https://github.com/JeffersonLab/SRO-RTDP/tree/main/src/utilities/cpp/podio2tcp

In [5]:
server_node.upload_file('simout.1000.edmhep.root.podiostr', 'simout.1000.edmhep.root.podiostr')

<SFTPAttributes: [ size=616565288 uid=1000 gid=1001 mode=0o100664 atime=1713097225 mtime=1713097301 ]>

# Setup scripts to run test

Numerous environment variables need to be set up inside the docker container before running the software. This is easiest to do by just putting them in a script. Copy the `setenv.sh` script from here to each of the nodes.

The client node (the one running eicrecon and consuming events) will need to know the IP address of the server node. Copy this into the setenv.sh scripts on the remote node(s) so it is set as an environment variable that can be easily used in the eicrecon command.



In [13]:
# Get IP address of server
server_addr = server_node.get_interface(network_name=f'FABNET_IPv4_{server_node.get_site()}').get_ip_addr()
print(f'server_addr: {server_addr}')

# Upload scripts to server
server_node.upload_file('setenv.sh', 'setenv.sh')
server_node.upload_file('run_source.sh', 'run_source.sh')
server_node.execute("chmod +x run_source.sh", quiet=True)

# Upload scripts to workers
for site,node in worker_nodes.items():
    node.upload_file('setenv.sh', 'setenv.sh')

    # Append setting the PODIOHOST to the setenv.sh script
    cmd = f"echo \"export PODIOHOST={server_addr}\" >> setenv.sh"
    node.execute(cmd, quiet=False, output_file=f"{node.get_name()}.log");
    node.upload_file('run_eicrecon.sh', 'run_eicrecon.sh')
    node.execute("chmod +x run_eicrecon.sh", quiet=True)


server_addr: 10.143.4.2


# Run processes

At this point, it is probably easier to run the processes manually in separate terminals connected to each host. Grab the ssh commands for each node from the top of this notebook and establish a connection to each in separate terminals. The run docker like this:

On node1 (CERN):
~~~
docker run -it --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_source.sh
~~~

On node2 (WASH):
~~~
docker run -it --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh
~~~



In [33]:
threads = []

# kill any eicrecon processes on worker nodes
for site,node in worker_nodes.items():
    node.execute("pkill -9 -f \"/work/run_eicrecon.sh\"", quiet=False)

# Start server
# thr = server_node.execute_thread("docker run -u rocky:rocky --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_source.sh")
# threads.append(thr)

# Start eicrecon workers
Ncores = 4
for site,node in worker_nodes.items():
    for i in range(Ncores):
        cmd = f"docker run -u 1000:1001 --rm --network host -v ${{PWD}}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh {i}"
        print(f"starting thread: {site}_{i} [] {cmd}")
        thr = node.execute_thread(cmd)
        threads.append(thr)

print(f"Num worker threads started: {len(threads)}")

starting thread: NEWY_0 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 0
starting thread: NEWY_1 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 1
starting thread: NEWY_2 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 2
starting thread: NEWY_3 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 3
starting thread: WASH_0 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 0
starting thread: WASH_1 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 1
starting thread: WASH_2 [] docker run -u 1000:1001 --rm --network host -v ${PWD}:/work eicweb/jug_xl:nightly /work/run_eicrecon.sh 2
starting thread: WASH_3 [] docker run -u 1000:1001 --rm --network hos

In [31]:

# stdout,stderr = threads[0].result()
# print(stdout)
# print(stderr)
    
# kill any eicrecon processes on worker nodes
for site,node in worker_nodes.items():
    print(f"{site}")
    node.execute("ps -efwww | grep \"/bin/bash /work/run_eicrecon.sh\" | grep -v grep", quiet=False)
    node.execute("pkill -2 -f \"/work/run_eicrecon.sh\"", quiet=False)
    node.execute("pkill -2 -f \"/work/run_eicrecon.sh\"", quiet=False)
    node.execute("pkill -2 -f \"/work/run_eicrecon.sh\"", quiet=False)
    node.execute("pkill -9 -f \"/work/run_eicrecon.sh\"", quiet=False)    

NEWY
rocky      20292   20269  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 0
rocky      20387   20366  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 2
rocky      20420   20400  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 3
rocky      20511   20478  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 1
WASH
rocky      20062   20042  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 2
rocky      20222   20184  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 3
rocky      20319   20273  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 0
rocky      20569   20538  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 1
ATLA
rocky      20810   20785  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 1
rocky      20846   20827  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 2
rocky      21157   21134  0 15:26 ?        00:00:00 /bin/bash /work/run_eicrecon.sh 3
rocky      21191   21170  0 15:26 ?    

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [34]:
# slice = fablib.get_slice(slice_name)
# slice.delete()
